In [18]:
import numpy as np
import gym
from gym import spaces
import random
import pandas as pd

**action**:
* Đánh 1 cây cùng chất
* Đánh 2
* Đánh 1 đôi
* Đánh 1 bộ ba
* Đánh tứ quý

* Đánh dây 3
* Đánh dây 4
* Đánh dây 5
* Đánh dây 6
* Đánh dây 7
* Đánh dây 8
* Đánh dây 9
* Đánh dây 10
* Đánh dây 11
* Đánh dây 12
* Đánh dây 13

* Bỏ lượt

In [17]:
data_cards = ['3_bich', '3_nhep', '3_do', '3_co' , '4_bich', '4_nhep', '4_do', '4_co', '5_bich', '5_nhep', '5_do', '5_co', '6_bich', '6_nhep', '6_do', '6_co',  '7_bich', '7_nhep', '7_do', '7_co',  '8_bich', '8_nhep', '8_do', '8_co', '9_bich', '9_nhep', '9_do', '9_co', '10_bich', '10_nhep', '10_do', '10_co', 'j_bich', 'j_nhep', 'j_do', 'j_co',  'q_bich', 'q_nhep', 'q_do', 'q_co', 'k_bich', 'k_nhep', 'k_do', 'k_co',  'A_bich', 'A_nhep', 'A_do', 'A_co', '2_bich', '2_nhep', '2_do', '2_co']

In [22]:
cards_data = pd.DataFrame(data_cards)
cards_data

,0
0,3_bich
1,3_nhep
2,3_do
3,3_co
4,4_bich
5,4_nhep
6,4_do
7,4_co
8,5_bich
9,5_nhep


In [14]:
class TLMB_Env(gym.Env):
    def __init__(self):
        super(TLMB_Env, self).__init__()
        self.num_cards = 52
        self.num_players = 4
        self.num_cards_per_player = 13
        
        self.action_space = spaces.Discrete(17)    #num_actions = 17
        self.observation_space = spaces.Dict({
            'player_hand': spaces.MultiBinary(self.num_cards),     
            'opponent_hand': spaces.MultiBinary((self.num_players - 1, self.num_cards)),
            'played_cards': spaces.MultiBinary(self.num_cards)
            'current_played_cards': spaces.MultiBinary(self.num_cards)
        })
        self.actions = [
            'play_single',
            'play_two'
            'play_pair',
            'play_trio',
            'play_four_of_a_kind',
            'play_straight_3',
            'play_straight_4',
            'play_straight_5',
            'play_straight_6',
            'play_straight_7',
            'play_straight_8',
            'play_straight_9',
            'play_straight_10',
            'play_straight_11',
            'play_straight_12',
            'play_straight_13',
            'pass_turn'
        ]
    def reset(self):
        deck = np.arange(self.num_cards)
        np.random.shuffle(deck)
        self.player_hands = [np.zeros(self.num_cards, dtype=np.int8) for _ in range(self.num_players)]
        for i in range(self.num_players):
            self.player_hands[i][deck[i * self.num_cards_per_player:(i + 1) * self.num_cards_per_player]] = 1
        
        self.played_cards = np.zeros(self.num_cards, dtype=np.int8)
        
        self.current_player = 0
        self.current_reward = [0 for _ in range(self.num_players)]
        self.current_played_cards =  [0 for _ in range(self.num_cards)]
        
        observation = self._get_obs()
        info = self._get_info()
        
        return observation,info
    def _get_observation(self):
        return {
            'player_hand': self.player_hands[0],     ## set train object is first player
            'opponent_hands': np.array([self.player_hands[i] for i in range(1,self.num_players)]),
            'played_cards': self.played_cards
        }
    def step(self, action):
        action_type = self.actions[action]
        curr_p = self.current_player
        
        reward = 0
        done = False
        
        if action_type == 'play_single':                                #Đánh 1 cây
            if np.sum(self.current_played_cards) == 1:
                idx_card_played = [i for i, value in enumerate(self.current_played_cards) if value == 1]
                idx_card_to_play = np.argmax(self.player_hands[curr_p][idx_card_played+1:] == 1) + idx_card_played + 1
                self.player_hands[idx_card_play] = 0
                self.current_played_cards[idx_card_play] = 1
                reward = 1
            
        elif action_type == 'play_two':                                  #Đánh 2
            if np.sum(self.current_played_cards) == 1:
                idx_card_play = np.argmax(self.player_hands[48:] == 1)      
                self.player_hands[idx_card_play] = 0
                self.current_played_cards[idx_card_play] = 1
                reward = 1
            
        elif action_type == 'play_pair':
            reward = 2
            
        elif action_type == 'play_trio':
            reward = 3
            
        elif action_type == 'play_four_of_a_kind':
            reward = 4
            
        elif action_type == 'play_straight_3':
            reward = 3
            
        elif action_type == 'play_straight_4':
            reward = 4
            
        elif action_type == 'play_straight_5':
            reward = 5
            
        elif action_type == 'play_straight_6':
            reward = 6
            
        elif action_type == 'play_straight_7':
            reward = 7
            
        elif action_type == 'play_straight_8':
            reward = 8
            
        elif action_type == 'play_straight_9':
            reward = 9
            
        elif action_type == 'play_straight_10':
            reward = 10
            
        elif action_type == 'play_straight_11':
            reward = 11
            
        elif action_type == 'play_straight_12':
            reward = 12
            
        elif action_type == 'play_straight_13':
            reward = 13
            
        
        self.current_player = (self.current_player + 1) % self.num_players
        done = self.check_game_over()
        
        if done:
            reward = self._calculate_final_rewards()

        return self._get_observation(), reward, done, {}
        
        
    def check_game_over(self):
        return any(np.sum(hand) == 0 for hand in self.player_hands)

    def _calculate_final_rewards(self):
        rewards = [-10 * np.sum(hand) for hand in self.player_hands]
        rewards[self.current_player] += 100 
        return rewards[self.current_player]
    
